In [1]:
import requests
import json
import base64

base_url = 'http://localhost:8081/v1/pytorch_api/'
detect_endpoint = 'detect'

async_url = base_url + detect_endpoint
# https://stackoverflow.com/questions/3715493/encoding-an-image-file-with-base64
def read_tif_as_base64_str(tif_path):
    ENCODING = 'utf-8'
    with open(tif_path, "rb") as image_file:
        gtiff_bytes = image_file.read()
        gtiff_bytes_b64_encoded = base64.b64encode(gtiff_bytes)
        return gtiff_bytes_b64_encoded.decode(ENCODING)
tif_path = "../images/aoi_restricted_LT05_CU_012008_20050702_20190102_C01_V0_-632777_2027019.tif"
payload = {'data': read_tif_as_base64_str(tif_path),
           'outname': "results_LT05_CU_012008_20050702_20190102_C01_V0_-632777_2027019.tif"
            }
# r = requests.post(async_url, data=read_tif_as_base64(tif_path))
r = requests.post(async_url, json=payload)
print(r.text)

TaskId: 5777


In [2]:
taskid = int(r.text.split(": ")[1])
url = f"http://localhost:8081/v1/pytorch_api/task/{taskid}"

payload={}
headers = {}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)


{
    "TaskId": 9078,
    "Status": "Task failed - try again",
    "Timestamp": "2021-01-06 05:48:18",
    "Endpoint": "uri"
}



### Server Code

In [3]:
gtiff_bytes_str = read_tif_as_base64_str(tif_path)

In [ ]:
gtiff_bytes_str

In [6]:
from rasterio.io import MemoryFile
from rasterio.plot import reshape_as_image

decoded_bytes = base64.b64decode(gtiff_bytes_str)
with MemoryFile(decoded_bytes) as memfile:
    with memfile.open() as src:
        meta = src.meta
        arr = reshape_as_image(src.read())



In [12]:
meta

{'driver': 'GTiff',
 'dtype': 'int16',
 'nodata': -9999.0,
 'width': 128,
 'height': 128,
 'count': 3,
 'crs': CRS.from_wkt('PROJCS["Albers",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378140,298.256999999996,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",23],PARAMETER["longitude_of_center",-96],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'),
 'transform': Affine(30.0, 0.0, -632777.949033806,
        0.0, -30.0, 2027019.8589219595)}